In [1]:
import tensorflow as tf
import json
import gzip
import numpy as np
from tqdm import tqdm_notebook

In [2]:
sess = None

In [3]:
def reset_tf(sess = None, log_device_placement = False):
    if sess:
        sess.close()
    tf.reset_default_graph()
    tf.set_random_seed(0)
    return tf.InteractiveSession(config = tf.ConfigProto(log_device_placement = log_device_placement))

In [4]:
class HyperParameters():
    # adam learning rate
    learning_rate = 1e-3

    # number of distinct terms (term indices are expected in 0..range(num_terms))
    vocab_size = 30000
    
    # number of dimensions in hidden layer
    hidden_size = 512

    # number of dimensions in document embedding
    embedding_size = 128
    
    # dropout rate
    dropout_rate = 0.1
    
    # number of sequences per batch
    pipeline_batch_size = 32
    
    # number of parsing threads in data pipeline
    pipeline_num_parallel_calls = 4
    
    # size of prefetch in data pipeline
    pipeline_prefetch_size = pipeline_batch_size * 16
    
    # shuffle buffer size
    pipeline_shuffle_size = 256

hp = HyperParameters()

In [5]:
sess = reset_tf(sess)

# Pipeline
# --------

# TODO: use SparseTensor / don't use dataset API for speed?

def parse_example(example_proto):
    features = {
        'page_id': tf.FixedLenFeature([1], dtype=tf.int64),
        'para_id': tf.FixedLenFeature([1], dtype=tf.int64),
        'indices': tf.VarLenFeature(tf.int64),
        'freqs': tf.VarLenFeature(tf.int64)
    }
    parsed = tf.parse_single_example(example_proto, features)
    page_id = parsed['page_id']
    para_id = parsed['para_id']
    indices = tf.sparse_tensor_to_dense(parsed['indices'])
    freqs = tf.sparse_tensor_to_dense(parsed['freqs'])
    return page_id, para_id, tf.cast(tf.sparse_to_dense(indices, [hp.vocab_size], freqs), tf.float32)

dataset_filenames = tf.placeholder(tf.string, shape = [None], name = 'dataset_filenames')

dataset = tf.data.TFRecordDataset(dataset_filenames)
dataset = dataset.map(parse_example,
                      num_parallel_calls = hp.pipeline_num_parallel_calls)
dataset = dataset.shuffle(hp.pipeline_shuffle_size)
dataset = dataset.prefetch(hp.pipeline_prefetch_size)
dataset = dataset.batch(hp.pipeline_batch_size)

dataset_iterator = dataset.make_initializable_iterator()

input_page_id_iter, input_para_id_iter, input_tf_vector_iter = dataset_iterator.get_next()

input_page_id = tf.placeholder_with_default(input_page_id_iter, [None, 1], name = 'input_page_id')
input_para_id = tf.placeholder_with_default(input_para_id_iter, [None, 1], name = 'input_para_id')
input_tf_vector = tf.placeholder_with_default(input_tf_vector_iter, 
                                              [None, hp.vocab_size],
                                              name = 'input_tf_vector')
input_tf_vector_count = tf.shape(input_tf_vector)[0]

input_tf_vector_norm = tf.reduce_sum(input_tf_vector, axis = -1,  keep_dims = True)
input_tf_vector_normalized = input_tf_vector / (input_tf_vector_norm + 1e-8)

# Model
# -----

def layer_dense_with_norm(x, num_units, scope, reuse=None, epsilon=1e-6):
    x = tf.layers.dense(x, num_units, activation = tf.nn.relu, name=scope)
    return x

layer = input_tf_vector_normalized

layer = layer_dense_with_norm(layer, hp.hidden_size, 'input_hidden_layer')
layer = layer_dense_with_norm(layer, hp.embedding_size, 'input_embedding_layer')
layer = layer_dense_with_norm(layer, hp.hidden_size, 'output_hidden_layer')

with tf.variable_scope('output_layer'):
    output_tf_vector_normalized = tf.layers.dense(layer,hp.vocab_size)

# Loss
# ----

indiv_loss = tf.losses.mean_squared_error(input_tf_vector_normalized,
                                          output_tf_vector_normalized,
                                          reduction = tf.losses.Reduction.NONE)
total_loss = tf.reduce_sum(indiv_loss, name = 'total_loss')
mean_loss = tf.div(total_loss, 
                   (tf.cast(input_tf_vector_count, tf.float32) * hp.vocab_size),
                   name = 'mean_loss')

# Optimization
# ------------

optimizer = tf.train.AdamOptimizer(learning_rate = hp.learning_rate)
train_op = optimizer.minimize(mean_loss)
# gradients, variables = zip(*optimizer.compute_gradients(mean_loss))
# gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
# train_op = optimizer.apply_gradients(zip(gradients, variables))

# Stats
# -----

total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    print('parameters for "%s": %d' % (variable.name, variable_parameters))
    total_parameters += variable_parameters
print('total parameters: %d' % total_parameters)

parameters for "input_hidden_layer/kernel:0": 15360000
parameters for "input_hidden_layer/bias:0": 512
parameters for "input_embedding_layer/kernel:0": 65536
parameters for "input_embedding_layer/bias:0": 128
parameters for "output_hidden_layer/kernel:0": 65536
parameters for "output_hidden_layer/bias:0": 512
parameters for "output_layer/dense/kernel:0": 15360000
parameters for "output_layer/dense/bias:0": 30000
total parameters: 30882224


In [6]:
sess.run(tf.global_variables_initializer())

In [7]:
def evaluate_dataset(dataset_filename,
                     header = 'results',
                     train = False,
                     show_progress = True):
    cum_loss = 0
    cum_count = 0

    sess.run(dataset_iterator.initializer, feed_dict={
        dataset_filenames: [dataset_filename]
    })

    if show_progress:
        progress = tqdm_notebook()

    while True:
        try:
            (_,
             curr_loss,
             curr_count) = sess.run((train_op if train else [],
                                     total_loss,
                                     input_tf_vector_count))
        except tf.errors.OutOfRangeError:
            break

        if show_progress:
            progress.update(curr_count)

        cum_loss += curr_loss
        cum_count += curr_count

    if show_progress:
        progress.close()

    print('%s: loss=%g (%g/%d)' % (header, cum_loss/cum_count, cum_loss, cum_count))

In [8]:
for epoch in range(50):
    evaluate_dataset('../data/simplewiki/simplewiki-20171103.topic_model.30k.train.tfrecords',
                     header = 'train %d' % epoch,
                     train = True,
                     show_progress = True)
    evaluate_dataset('../data/simplewiki/simplewiki-20171103.topic_model.30k.dev.tfrecords',
                     header = 'dev   %d' % epoch,
                     train = False,
                     show_progress = False)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


train 0: loss=0.0250402 (2152.13/85947)
dev   0: loss=0.0196415 (392.831/20000)


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


train 1: loss=0.0181709 (1561.74/85947)
dev   1: loss=0.0165593 (331.185/20000)


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

...


train 47: loss=0.0105567 (907.317/85947)
dev   47: loss=0.0107332 (214.665/20000)


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


train 48: loss=0.0105383 (905.735/85947)
dev   48: loss=0.0107166 (214.333/20000)


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


train 49: loss=0.0105216 (904.304/85947)
dev   49: loss=0.0107015 (214.029/20000)


In [9]:
builder = tf.saved_model.builder.SavedModelBuilder('../models/simplewiki/topic_model_1_128')
builder.add_meta_graph_and_variables(sess, ['training'])
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../models/simplewiki/topic_model_1_128/saved_model.pb'


b'../models/simplewiki/topic_model_1_128/saved_model.pb'

In [6]:
# sess = reset_tf()
# tf.saved_model.loader.load(sess, ['training'], '../models/simplewiki/topic_model_1_256')
# print('loaded')

INFO:tensorflow:Restoring parameters from b'../models/simplewiki/topic_model_1_256/variables/variables'
loaded


In [7]:
# dataset_filenames = tf.get_default_graph().get_operation_by_name('dataset_filenames').outputs[0]
# embedding_layer = tf.get_default_graph().get_operation_by_name('input_embedding_layer/Relu').outputs[0]
# input_page_id = tf.get_default_graph().get_operation_by_name('input_page_id').outputs[0]
# input_para_id = tf.get_default_graph().get_operation_by_name('input_para_id').outputs[0]
# make_iterator = tf.get_default_graph().get_operation_by_name('MakeIterator')

In [10]:
make_iterator = dataset_iterator.initializer
embedding_layer = tf.get_default_graph().get_operation_by_name('input_embedding_layer/Relu').outputs[0]

In [14]:
def extract_embeddings(dataset_filename):
    sess.run(make_iterator, feed_dict={
        dataset_filenames: [dataset_filename]
    })
    
    progress = tqdm_notebook()
    
    result = []
    
    while True:
        try:
            (curr_input_page_id, 
             curr_input_para_id, 
             curr_embedding_layer) = sess.run((input_page_id, input_para_id, embedding_layer))
        except tf.errors.OutOfRangeError:
            break
        for i in range(curr_input_page_id.shape[0]):
            page_id = int(curr_input_page_id[i][0])
            embedding = curr_embedding_layer[i].tolist()
            result.append((page_id, embedding))
            
        progress.update(curr_input_page_id.shape[0])
    
    progress.close()
    
    return result

In [15]:
all_embeddings = []

all_embeddings.extend(extract_embeddings('../data/simplewiki/simplewiki-20171103.topic_model.30k.dev.tfrecords'))
all_embeddings.extend(extract_embeddings('../data/simplewiki/simplewiki-20171103.topic_model.30k.test.tfrecords'))
all_embeddings.extend(extract_embeddings('../data/simplewiki/simplewiki-20171103.topic_model.30k.train.tfrecords'))

all_embeddings.sort()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [21]:
for i, (j, _) in enumerate(all_embeddings):
    assert i == j

In [27]:
all_embeddings = np.array([v for _, v in all_embeddings])

In [30]:
with gzip.open('../data/simplewiki/simplewiki-20171103.topic_model_1_128.embedding.npy.gz', 'wb') as f:
    np.save(f, all_embeddings)